In [1]:
!pip install sentence-transformers phi3 mistral


ERROR: Could not find a version that satisfies the requirement phi3 (from versions: none)
ERROR: No matching distribution found for phi3


In [3]:
!pip install phi3

ERROR: Could not find a version that satisfies the requirement phi3 (from versions: none)
ERROR: No matching distribution found for phi3


In [4]:
from sentence_transformers import SentenceTransformer, util
import phi3
import os
import json

# Load embedding models
model_layer1 = SentenceTransformer('all-MiniLM-L6-v2')
model_layer2 = SentenceTransformer('all-MPNet-base-v2')

# Load dataset
dataset_path = "C:/Users/jamal/OneDrive/Desktop/Company_Assessment/Evvo Technology/dme_deloitte-global-minimum-tax-faq.pdf"
with open(dataset_path, 'r') as f:
    documents = json.load(f)

# Create vector stores
vector_store_layer1 = phi3.VectorStore()
vector_store_layer2 = phi3.VectorStore()

# Process and store keywords/summaries (Layer 1)
for doc_id, doc in documents.items():
    summary = doc['summary']  # Assuming each document has a summary
    embedding = model_layer1.encode(summary, convert_to_tensor=True)
    vector_store_layer1.add(doc_id, embedding)

# Process and store full documents (Layer 2)
for doc_id, doc in documents.items():
    full_text = doc['text']  # Full document text
    embedding = model_layer2.encode(full_text, convert_to_tensor=True)
    vector_store_layer2.add(doc_id, embedding)

# Save vector stores for later use
vector_store_layer1.save('vector_store_layer1')
vector_store_layer2.save('vector_store_layer2')

# Function to retrieve documents
def retrieve_documents(query):
    # Retrieve summaries (Layer 1)
    query_embedding_layer1 = model_layer1.encode(query, convert_to_tensor=True)
    results_layer1 = vector_store_layer1.search(query_embedding_layer1, top_k=10)
    
    # Retrieve full documents (Layer 2)
    relevant_doc_ids = [result['id'] for result in results_layer1]
    full_docs_embeddings = [vector_store_layer2.get(doc_id) for doc_id in relevant_doc_ids]
    query_embedding_layer2 = model_layer2.encode(query, convert_to_tensor=True)
    
    results_layer2 = util.semantic_search(query_embedding_layer2, full_docs_embeddings, top_k=3)
    return results_layer2

# Example query
query = "Climate change impact on agriculture"
results = retrieve_documents(query)
print(results)


ModuleNotFoundError: No module named 'phi3'